## Analyse des données public sur l'épidémie de covid19 en France

Source: [https://www.data.gouv.fr/en/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/]

Données du 14 Octobre

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('./data/donnees-hospitalieres-classe-age-covid19-2020-10-23-19h00.csv', sep=';',parse_dates=['jour'])

## Analyse temporelle par classe d'âge

Analyse de la dynamique des décès par classe d'àge.

In [27]:
import numpy as np
import pandas_bokeh as pb

In [28]:
pb.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')
# Create Bokeh-Table with DataFrame:
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource

Loading BokehJS ...

In [29]:
deces_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='dc',aggfunc='sum')
deces_par_age.plot_bokeh(kind="line",title ="Covid - répartition des décès cumulés",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Décès')

Figure(id='3612', ...)

## Analyse de la situation actuelle

Analyse simplifiée retenant deux groupes d'àge:
* moins de 60 ans
* plus de 60 ans

In [30]:
deces_derniere_situation = df[df['cl_age90']>0].drop(['hosp','rea','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()

In [31]:

deces_moins_60 = deces_derniere_situation[deces_derniere_situation['cl_age90'] < 69]['dc'].sum()
deces_plus_60 = deces_derniere_situation[deces_derniere_situation['cl_age90'] >= 69]['dc'].sum()

In [32]:
pd.DataFrame(np.array([[deces_moins_60], [deces_plus_60]]),
                   columns=['deces'],index=['moins de 60','plus de 60']).plot_bokeh(kind="pie",title ="Covid - répartition des décès cumulés",
                   figsize =(600,600),xlabel='groupe d\'àge')

__x__values_original


Figure(id='4288', ...)

In [33]:
deces_derniere_situation.drop(['jour'],axis=1).plot_bokeh.bar(
    x='cl_age90',
    y='dc',
    title="Impact du covid19 par catégorie d'àge", 
    alpha=0.6,
    xlabel="catégorie d'àge",
    ylabel="deces" )

Figure(id='4461', ...)

## Analyse comparée avec la démographie française

[https://www.insee.fr/fr/statistiques/4515539?sommaire=4516122]


In [34]:
demo = pd.read_csv('data/BTT_TD_POP1A_2017.CSV',sep=';')
demo_agg = demo.drop(['NIVGEO','CODGEO','LIBGEO','SEXE'],axis=1).groupby('AGEPYR10').agg('sum')


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Basé sur le dernier recensement

* calcul de la population totale
* calcul de la population des moins de 55 ans
* calcul de la population des plus de 55 ans



In [35]:
nb_total = demo['NB'].sum()
nb_moins_55 = demo_agg.loc[0:55].sum()['NB']
nb_plus_55 = demo_agg.loc[65:].sum()['NB']

In [36]:
print("population total: {:.0f}".format(nb_total))
print("population moins de 55 ans {:.0f}".format(nb_moins_55))
print("population plus de 55 ans {:.0f}".format(nb_plus_55))

population total: 70091267
population moins de 55 ans 56567511
population plus de 55 ans 13523756


In [37]:
res_agg = pd.DataFrame(np.array([[nb_moins_55,deces_moins_60], [nb_plus_55,deces_plus_60]]),
                   columns=['population','deces'],index=['moins de 55/60','plus de 55/60']).astype(int)
stacked_bar = res_agg.plot_bokeh.bar(
    ylabel="population", 
    title="Impact du covid19 par catégorie d'àge", 
    alpha=0.6,
    xlabel='categorie')

### Conclusion

Les plus de 60 ans sont clairement plus à risque: sur 20000 morts à l'hôpital, 19000 ont plus de 60 ans.


## Décès quotidiens par tranche d'âge 

In [51]:
deces_par_age.diff().rolling(7).mean().plot_bokeh(kind="line",title ="Covid -  décès quotidiens par classe d'âge",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Décès quotidens')

Figure(id='6992', ...)

In [49]:
deces_par_age.diff()

cl_age90,9,19,29,39,49,59,69,79,89,90
jour,,,,,,,,,,
2020-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-19,0.0,0.0,1.0,0.0,1.0,6.0,16.0,36.0,72.0,18.0
2020-03-20,0.0,0.0,-1.0,0.0,0.0,2.0,6.0,24.0,37.0,10.0
2020-03-21,0.0,0.0,0.0,0.0,0.0,4.0,11.0,30.0,46.0,19.0
2020-03-22,0.0,0.0,0.0,1.0,3.0,0.0,15.0,28.0,42.0,23.0
...,...,...,...,...,...,...,...,...,...,...
2020-10-19,0.0,0.0,0.0,0.0,3.0,5.0,12.0,22.0,71.0,31.0
2020-10-20,0.0,0.0,0.0,1.0,2.0,6.0,12.0,35.0,62.0,44.0
2020-10-21,0.0,0.0,0.0,2.0,3.0,5.0,13.0,43.0,49.0,48.0


## Situation en réanimation

In [38]:
rea_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='rea',aggfunc='sum')
rea_par_age.plot_bokeh(kind="line",title ="Covid - patients en réanimation",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Patients en réanimation')

Figure(id='4790', ...)

In [39]:
rea_derniere_situation = df[df['cl_age90']>0].drop(['hosp','dc','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()
rea_moins_60 = rea_derniere_situation[rea_derniere_situation['cl_age90'] < 69]['rea'].sum()
rea_plus_60 = rea_derniere_situation[rea_derniere_situation['cl_age90'] >= 69]['rea'].sum()

In [40]:
rea_moins_60

521

In [41]:
rea_plus_60

1899

In [42]:
rea_moins_60 / (rea_moins_60 + rea_plus_60)

0.2152892561983471

### Conclusion

22% des patients en réanimation ont moins de 60 ans au 14 Octobre 2020

## Situation à l'hôpital

In [43]:
hosp_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='hosp',aggfunc='sum')
hosp_par_age.plot_bokeh(kind="line",title ="Covid - patients hospitalisés",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Patients hospitalisés')

Figure(id='5487', ...)

In [44]:
hosp_derniere_situation = df[df['cl_age90']>0].drop(['rea','dc','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()
hosp_moins_60 = hosp_derniere_situation[hosp_derniere_situation['cl_age90'] < 69]['hosp'].sum()
hosp_plus_60 = hosp_derniere_situation[hosp_derniere_situation['cl_age90'] >= 69]['hosp'].sum()

In [45]:
hosp_plus_60

12365

In [46]:
hosp_moins_60

2542

In [47]:
hosp_moins_60/(hosp_moins_60+hosp_plus_60)

0.17052391493929026

### Conclusion

17% des patients hospitalisés ont moins de 60 ans au 14 Octobre 2020